<a href="https://colab.research.google.com/github/annvix/Accents_LSTM-/blob/master/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Клонирование проекта с GitHub и подготовка методов
Подключение библиотек

Загрузка предобученной модели

In [1]:
!git clone https://github.com/annvix/Accents_LSTM-.git

Cloning into 'Accents_LSTM-'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 62 (delta 26), reused 34 (delta 8), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [0]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import random
import numpy as np

model = load_model('Accents_LSTM-/model.h5')
#model = load_model('model.h5')
model.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False, name='categorical_crossentropy'),
                  metrics=['accuracy'])

Добавляем алфавит для кодирования букв

In [0]:
alphabet =   {'а' : 1, 'б' : 2, 'в' : 3, 'г' : 4, 'д' : 5, 'е' : 6, 'ё' : 7, 'ж' : 8, 'з' : 9,
             'и' : 10, 'й' : 11, 'к' : 12, 'л' : 13, 'м' : 14, 'н' : 15, 'о' : 16, 'п' : 17, 'р' : 18,
             'с' : 19, 'т' : 20, 'у' : 21, 'ф' : 22, 'х' : 23, 'ц' : 24, 'ч' : 25, 'ш' : 26, 'щ' : 27,
             'ъ' : 28, 'ы' : 29, 'ь' : 30, 'э' : 31, 'ю' : 32, 'я' : 33}

vovels = ['а', 'о', 'э', 'и', 'у', 'ы', 'е', 'ё', 'ю', 'я']
max_len = 29

Метод для возвращения ключа по значению

In [0]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

Метод для проверки, есть ли гласные в слове -> есть ли ударение

In [0]:
def is_acc (w):
    for i in w:
        if i in vovels: 
            return True
            break
    return False

Метод для удаления спец. символов из предложения

In [0]:
from string import punctuation

def text_without_punct(text):
    return ''.join([i for i in text if i not in punctuation])

# Постановка ударений

Постановка ударения для одного слова

In [6]:
word = 'тараканище'
#word = 'бррр'
if is_acc(word):
  word_int = np.zeros((max_len), dtype=int)

  for i in range(len(word)):
    word_int[i]= alphabet[word[i]]
    
  word_int = np.reshape(word_int, [1,29])
  acc_pred = model.predict(word_int)

  for i in range(1):
    slovo = ''
    for j in range(len(word_int[i])):
        if word_int[i][j] != 0:
            slovo = slovo + get_key(alphabet, word_int[i][j])
        else:
            break
    nn_sample = slovo[:acc_pred[i].argmax()+1] + '\'' + slovo[acc_pred[i].argmax()+1:]
    print('слово    : ' , slovo)
    print('Результат: ' , nn_sample)
else:
  print('В этом слове нет ударения:', word)

слово    :  тараканище
Результат:  тарака'нище


Метод для постановки ударения для предложения

In [0]:
def acc_in_sent(str):
    
    # перевод предложения в массив слов из закодированных букв
    no_punct_text = text_without_punct(str)
    str_ls = ''.join(no_punct_text).lower()
    str_ls = str_ls.split()
    str_sent_int = np.zeros((len(str_ls),max_len), dtype=int)
    for i in range(len(str_ls)):
        for j in range(len(str_ls[i])):
            str_sent_int[i][j]= alphabet[str_ls[i][j]]

    # предсказывание ударения
    acc_pred = model.predict(str_sent_int)

    #Вывод слов с ударениями в необходимом формате
    final_sent_ = str.split()
    final = ''
    for i in range(len(str_ls)):
      if is_acc(str_ls[i]):
        final = final + final_sent_[i][:acc_pred[i].argmax()+1] + '\'' + final_sent_[i][acc_pred[i].argmax()+1:] + ' '
      else:
        final = final + final_sent_[i][:acc_pred[i].argmax()+1] + '' + final_sent_[i][acc_pred[i].argmax()+1:] + ' '
    
    print (final)
    return final

Постановка ударения на тестовом предложении

In [8]:
test = 'В России зимой бывает много снега.'
my_test = acc_in_sent(test)

В Ро'ссии зи'мой быва'ет мно'го сне'га. 


Постановка ударения для списка тестовых предложений из файла

In [0]:
with open('Accents_LSTM-/sentences.txt', 'r') as f:
#with open('sentences.txt', 'r') as f:
    sent_ = f.readlines()

In [0]:
random.shuffle(sent_)

sent = sent_

no_punct_text = text_without_punct(sent)
sent_split = no_punct_text.split('\n')

test_sent = []
for i in range(10):
    test_sent.append(sent_split[i])

In [11]:
for k in range(len(test_sent)):
    acc_in_sent(test_sent[k])
    print()

Я' изба'вился о'т мра'чных мы'слей. 

О'на поспе'шно напра'вилась к телефо'нному аппара'ту. 

О'ни сно'ва поплели'сь за' ко'шкой. 

О'ни упа'ли в сне'г. 

Рича'рд вы'шел и'з маши'ны. 

О'на зарыча'ла о'т стра'ха. 

Хозя'йка хло'пнулась в обмо'рок. 

О'ни все'мером юти'лись в дровяно'м сара'е. 

О'н стро'го взгля'нул на' уче'ного. 

Ди'нка сиде'ла в кле'тке. 

